#### Install Dependencies

In [ ]:
#@markdown #### Install Dependencies
%%capture
!wget https://github.com/mitkeng/AGENTi/raw/refs/heads/main/multi-models/agentI_anom_model.zip
!wget https://github.com/mitkeng/AGENTi/raw/refs/heads/main/multi-models/agentI_reg_model.zip
!wget https://github.com/mitkeng/AGENTi/raw/refs/heads/main/multi-models/autoencoder_model.keras



!pip install rdkit
!pip install -q condacolab
!pip install ydf -U -qq
!pip install tensorflow_decision_forests -U -qq
!pip install trainer
!pip install PubChemPy
!pip install cirpy
!pip install ase
import condacolab
condacolab.install()
import condacolab



In [ ]:
%%capture
!pip install rdkit
from google.colab import output
output.enable_custom_widget_manager()


from tensorflow import keras
from numpy.core.arrayprint import printoptions
import tensorflow as tf
import numpy as np

import os
import tensorflow as tf


In [ ]:
%%capture
import cirpy
import pubchempy as pcp
from rdkit import Chem
import os
import pandas as pd
from urllib.request import urlopen
from pubchempy import get_compounds, Compound
import pubchempy as pcp
from rdkit import Chem
import os
import pandas as pd
import pandas as pd
import math
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sys
import re
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.python.ops.gen_dataset_ops import filter_dataset_eager_fallback
from tensorflow.python.ops.gen_math_ops import Max
from statistics import*
from urllib.request import urlopen
from urllib.parse import quote
from copy import deepcopy
import pubchempy as pcp
from rdkit.Chem import Descriptors
from rdkit import Chem
import os


## Enter Query System(s)

In [ ]:
#@markdown ##Enter Query System(s)

import csv
import os
import ase
import cirpy
import pubchempy as pcp
from rdkit import Chem
import os
import pandas as pd
from urllib.request import urlopen
from pubchempy import get_compounds, Compound
import pubchempy as pcp
from rdkit import Chem
import os
import pandas as pd
import pandas as pd
import math
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sys
import re
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.python.ops.gen_dataset_ops import filter_dataset_eager_fallback
from tensorflow.python.ops.gen_math_ops import Max
from statistics import*
from urllib.request import urlopen
from urllib.parse import quote
from copy import deepcopy
import pubchempy as pcp
from rdkit.Chem import Descriptors
from rdkit import Chem
import os
import pandas as pd
import numpy as np
from ase import io
from ase.io import read, write
import os
import shutil
import __main__
from google.colab import output
from google.colab import drive


def get_query():
  ex_count=0
  for q in os.listdir():
    fi_list = q.split(".")
    if 'csv' in fi_list and "protomers" not in fi_list and "results_data" not in fi_list:
      ex_count=+1
      return q
  if ex_count==0:
    return "ask_user"

try:
  os.remove("results_data.csv")
except:
  pass
try:
  os.remove("single_quiry.csv")
except:
  pass


input_query = get_query()
input_query2 = [input_query]


if "ask_user"  in input_query:
  file_name2 = "single_quiry.csv"
  query_file = open(file_name2,"w")
  input_query2=[input("Enter SMILE: ")]
  for l in range(len(input_query2)):

    print(input_query2[l].strip(), file=query_file)
  query_file.close()
  file_name = file_name2
else:
  file_name = input_query
  pass


smile_file = open(file_name)
smi_list_ = []

for fi in smile_file:
  if ".csv" in input_query:
    smi= fi.strip()
    smi_list_.append(smi.strip())
  else:
    smi=fi.strip()
    smi_list_= smi
    smi_list =[smi_list_]
if ".csv" in input_query:
  smi_list = [item for item in smi_list_ if item != ""]


## ML Protomer Shift Detection

In [ ]:
#@markdown ###Excute to continue

import math
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sys
import cirpy
import pubchempy as pcp
from rdkit import Chem
import os
import pandas as pd
from urllib.request import urlopen
from pubchempy import get_compounds, Compound
import pubchempy as pcp
from rdkit import Chem
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.python.ops.gen_dataset_ops import filter_dataset_eager_fallback
from tensorflow.python.ops.gen_math_ops import Max
from statistics import*

import zipfile

try:
  with zipfile.ZipFile('apciesi_anom_model.zip', 'r') as zip_:
    zip_.extractall('apciesi_anom_model')
except:
  pass
try:
  with zipfile.ZipFile('apciesi_reg_model.zip', 'r') as zip_:
    zip_.extractall('apciesi_reg_model')
except:
  pass

atom_radii = {"H":1.2, "C":1.7, "N":1.55, "O":1.52,"P": 1.8, "S": 1.8,"F": 1.35, "Cl":1.75, "Br":1.83}
atom_mass = {"H":1, "C":12, "N":14, "O":16,"P": 31, "S": 32,"F": 19, "Cl":35.34, "Br":79.9}
electro_neg= {"C":2.55, "O":3.44, "H": 2.2, "N": 3.04, "P": 2.19, "S": 2.58,"F": 3.98, "Cl":3.16, "Br":2.96}
ionize_E= {"C":11.26, "O":13.618, "H": 13.598, "N": 14.534, "P": 10.487, "S": 10.360,"F": 17.423, "Cl":12.968, "Br":11.8}
polari =  {"C":1.76, "O":0.802, "H": 0.667, "N": 1.1, "P": 3.63, "S": 2.9,"F": 0.557, "Cl":2.18, "Br":3.05}


test_file ="protomers.csv"


new_csv = open(test_file, "w")
print("smile,radii_mass2,electronic2,double_bond2,triple_bond2,hetero2,halogen,ring2,TPSA,VSA,logP,pos_charge,neg_charge", file = new_csv)



for mi in range(len(smi_list)):
  smile_list=[]
  try:
    for s in smi_list[mi]:
      s = s.strip().split('\n')

      s2 = s[0].strip(',')
      smile = s2.split(',')
      smile_list.append(smile[0])

    atom_list=[]

    for M in smile_list:
      halogen_count=0
      third_row = 0

      atom_list.append(M.strip())

      double_bond = atom_list.count("=")
      double_bond2= double_bond
      triple_bond = atom_list.count("#")
      triple_bond2=triple_bond
      third_row += atom_list.count("S")
      third_row += atom_list.count("P")
      halogen_count+= atom_list.count("l")
      halogen_count+= atom_list.count("F")
      halogen_count+= atom_list.count("r")
      halogen_count+= atom_list.count("I")
      molecule_set = set(atom_list)
      hetero_set = set(atom_list)
      try:
        molecule_set.remove('(')
        molecule_set.remove(')')
      except KeyError:
        next

      molecule_list = list(molecule_set)

      total_radii=[]
      total_mass=[]
      total_EN=[]
      total_IE=[]
      total_pol=[]
      total_hetero = []
      total_atom = []
      total_ring=[]

      for a in range(len(molecule_list)):

        b=atom_list.count(molecule_list[a])
        if molecule_list[a].isnumeric():
          all_ring=atom_list.count(molecule_list[a])
          total_ring.append(all_ring)
        try:
          atom_ra = atom_radii[molecule_list[a]]*b
          atom_ma = atom_mass[molecule_list[a]]*b
          el_neg = electro_neg[molecule_list[a]]*b

          ion_E = ionize_E[molecule_list[a]]*b
          polarizability = polari[molecule_list[a]]*b
          total_radii.append(atom_ra)
          total_mass.append(atom_ma)
          total_EN.append(el_neg)
          total_IE.append(ion_E)
          total_pol.append(polarizability)
        except KeyError:
          next

        if molecule_list[a].isalpha() and molecule_list[a] != "H" and molecule_list[a]!="C":
          hetero=atom_list.count(molecule_list[a])
          total_hetero.append(hetero)
        if molecule_list[a].isalpha():
          all_atom=atom_list.count(molecule_list[a])
          total_atom.append(all_atom)


      sm = Chem.MolFromSmiles(smi_list[mi])

      TPSA = round(Descriptors.TPSA(sm),2)
      VSA = float(round(Descriptors.VSA_EState2(sm),2))
      logP = round(Descriptors.MolLogP(sm),2)
      pos_charge = atom_list.count("+")
      neg_charge = atom_list.count("-")
      radii,mass,EN,IE,Polar, hetero=sum(total_radii),sum(total_mass),sum(total_EN),sum(total_IE),sum(total_pol), sum(total_hetero)
      hetero2 = hetero
      tot_atom = sum(total_atom)
      tot_ring = sum(total_ring)/2
      radii_mass2 =round((radii*tot_atom)/sum(total_mass),2)
      electronic2 = round((IE*EN)/(Polar*tot_atom ),2)
      ring2 = int(tot_ring)
      features=smi_list[mi],radii_mass2,electronic2,double_bond2,triple_bond2,hetero2,halogen_count,ring2,TPSA,VSA,logP,pos_charge, neg_charge

    new_csv = open(test_file, "a")
    print(features[0],",",features[1],",",features[2],",",features[3],",",features[4],",",features[5],",",features[6],",",features[7],",",features[8],",",features[9],",",features[10],",",features[11],",",features[12],file=new_csv)
    new_csv.close()
  except:
    pass






In [ ]:
#@markdown ###Excute to continue

import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler,Normalizer,RobustScaler,StandardScaler
import numpy as np
import pandas as pd


from tensorflow.keras.models import load_model
auto_mode = load_model("autoencoder_model.keras")

scaler = Normalizer()



df_TOTAL = pd.read_csv('protomers.csv')
df_TOTAL.pop('smile')
TOTAL_array = np.array(df_TOTAL)
scaled_TOTAL_data = scaler.fit_transform(TOTAL_array)
reconstruction_errors = np.mean(np.power(scaled_TOTAL_data - auto_mode.predict(scaled_TOTAL_data),2 ), axis=1)
st_dev = 0.0627
norm_mean = 0.0456
anomalies = [reconstruction_errors> (st_dev)]

count_anom = []

for i in anomalies[0].tolist():

  if str(i) =="True":
    count_anom.append("True")
  else:
    count_anom.append("False")



In [ ]:
#@markdown ##results
%%capture

import ydf

import zipfile
def unzip():
  for z in  os.listdir():
    if "zip" in z:
      zip_file_path = f'{z}'
      extract_to_path = f'{z.replace('.zip','')}'
      os.makedirs(extract_to_path, exist_ok=True)
      with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
          zip_ref.extractall(extract_to_path)


try:
  extract_zip = unzip()
except:
  pass

anom_model=ydf.load_model("agentI_anom_model")
reg_model=ydf.load_model("agentI_reg_model")

test_dat = pd.read_csv("protomers.csv")

prediction1 = anom_model.predict(test_dat)
prediction2 = reg_model.predict(test_dat)


index_list = []
anom_score = []
for p in range(len(prediction1)):
  if mean([prediction1[p],prediction2[p]])+reconstruction_errors[p]/st_dev>1.4:
    anom_score.append(round(mean([prediction1[p],prediction2[p]])+reconstruction_errors[p]/st_dev,2))
    index_list.append(int(str(test_dat[p:p+1]["smile"]).split()[0]))
    print('yes')





## Results

In [ ]:
#@markdown ### Results Preparation and Output
import csv
import pubchempy as pcp
import requests
def cactussearch2(ids):

      smile_search = pcp.get_compounds(ids, 'smiles')[0]
      cids = smile_search.cid

      compound_name = get_compound_name_from_cid(cids)
      return compound_name




def get_compound_name_from_cid(cid):
  """Retrieves the compound name from a PubChem CID using PUG REST."""
  url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/synonyms/TXT"
  response = requests.get(url)
  response.raise_for_status()
  synonyms = response.text.strip().split('\n')
  return synonyms


results_file = open("results_data.csv","w")

print("ChemID,SMILE,Threshold,A-Score",file=results_file)
results_file.close()
for i in range(len(index_list)):
  try:
    results_file = open("results_data.csv","a")
    nam = cactussearch2(test_dat.iloc[index_list[i]]["smile"])
    nam = [x.replace(",","") for x in nam]
    anom_smile = test_dat.iloc[index_list[i]]["smile"]
    print("{},{},1.40,{}".format(nam[0],anom_smile,anom_score[i]),file=results_file)
  except:
    pass

results_file.close()

results_show = pd.read_csv("results_data.csv")
results_show